# Aula 03 - Métodos de Boosting (Parte I)

Imaginemos a seguinte situação: precisamos modelar as vendas de um determinado produto para fazer previsões, o mais precisas quanto possível, das vendas por mês. Como poderíamos abordar este problema?

Em um primeiro momento, poderíamos, por exemplo, pensar em utilizar a média de vendas em um determinado período (por exemplo, nos últimos 12 meses) como o valor a ser predito todos os meses. No entanto, é evidente que a performance desta abordagem deixaria muito a desejar, já que estaríamos simplesmente "chutando" a média de vendas o tempo todo. Contudo, pode nos servir como ponto de partida, e levantar o seguinte questionamento: *e se utilizássemos os erros do modelo como novas informações para o treinamento de um outro modelo?**

Este é o princípio do **boosting:** utilizamos os erros no aprendizado de um modelo para treinar modelos subsequentes. Deste modo, a ideia é que "os modelos passem a aprender com os erros passados" e possa, assim, conjuntamente, performar melhor. Ou seja: *mesmo que a performance de cada um dos estimadores, individualmente, não seja boa, em conjunto, com o boosting, eles deveriam prover uma performance mais satisfatória!*

# "Trabalho em conjunto": Bagging x Boosting

Temos algumas maneiras de criar *comitês* de estimadores. A um grupo de modelos atuando conjuntamente, seja para fins de classificação ou regressão, chamamos comumente de **ensemble** (conjunto). A ideia de utilizar diversos estimadores conjuntamente tem por **objetivo combinar modelos mais simples em um único modelo mais robusto, a fim de reduzir o viés, a variância e/ou aumentar a acurácia**.

<div class="warning" style='padding:0.1em; background-color:#E9D8FD; color:#69337A'>
<span>
<h2>Algumas conceitualizações...</h2>
<ul>
<li>Podemos dizer que um modelo é um <b>weak learner</b> quando sua performance não é muito além de um "chute aleatório". <br><br>
    <li>Por outro lado, um <b>strong learner</b> possui alta performance e boa capacidade de generalização;<br><br>
<li>A ideia de métodos de ensemble é <b>combinar vários classificadores weak learners<\b> para gerar boas performances a custos computacionais mais baixos.
</ul>
</span>
<br>
</div>

## Alguns tipos de Ensemble:
- __1. Bagging (short for bootstrap aggregation)__: Treina paralelamente $N$ modelos mais fracos (geralmente do mesmo tipo) com $N$ subsets distintos criados com amostragem randômica e reposição. Cada modelo é avaliado na fase de teste com o label definido pela moda (classificação) ou pela média dos valores (regressão). Os métodos de Bagging reduzem a variância da predição. <br>
Algoritimos  famosos: Random Forest <br>
<img src='https://miro.medium.com/v2/resize:fit:828/format:webp/1*_pfQ7Xf-BAwfQXtaBbNTEg.png' style="width:600px"  text="https://miro.medium.com/v2/resize:fit:828/format:webp/1*_pfQ7Xf-BAwfQXtaBbNTEg.png" />  
<br>
<br>
- __2. Boosting__: Treina $N$ modelos mais fracos (geralmente do mesmo tipo) de **forma sequencial**. Os pontos que foram classificados erroneamente recebem um peso maior para entrar no próximo modelo. Na fase de teste, cada modelo é avaliado com base do erro de teste de cada modelo, e a predição é feita com um peso sobre a votação. Os métodos de Boosting reduzem o viés da predição. <br>
Algoritimos  famosos: AdaBoost, Gradient Boosting, XGBoost, CatBoost, LightGBM (Light Gradient Boosting Machine) <br>
<img src='https://media.geeksforgeeks.org/wp-content/uploads/20210707140911/Boosting.png' style="width:600px" text="Fonte: https://media.geeksforgeeks.org/wp-content/uploads/20210707140911/Boosting.png" />
<br>
<br>

##### Algumas leituras complementares:
[What is the difference between Bagging and Boosting?](https://quantdare.com/what-is-the-difference-between-bagging-and-boosting/)

[Bagging vs Boosting in Machine Learning](https://www.geeksforgeeks.org/bagging-vs-boosting-in-machine-learning/)


# Boosting : AdaBoost

O AdaBoost significa **Adaptive Boosting**, e tem como procedimento geral **a criação sucessiva dos chamados weak learners**, que são modelos bem fracos de aprendizagem - geralmente, **árvores de um único nó (stumps)**.

<img src="https://miro.medium.com/max/1744/1*nJ5VrsiS1yaOR77d4h8gyw.png" width=300>

O AdaBoost utiliza os **erros da árvore anterior para melhorar a próxima árvore**. As predições finais são feitas com base **nos pesos de cada stump**, cuja determinação faz parte do algoritmo!

<img src="https://static.packt-cdn.com/products/9781788295758/graphics/image_04_046-1.png" width=700>

Vamos entender um pouco melhor...

Aqui, o bootstrapping não é utilizado: o método começa treinando um classificador fraco **no dataset original**, e depois treina diversas cópias adicionais do classificador **no mesmo dataset**, mas dando **um peso maior às observações que foram classificadas erroneamente** (ou, no caso de regressões, a observações **com o maior erro**).

Assim, após diversas iterações, classificadores/regressores vão sequencialmente "focando nos casos mais difíceis", e construindo um classificador encadeado que seja forte, apesar de utilizar diversos classificadores fracos em como elementos fundamentais.

<img src="https://www.researchgate.net/profile/Zhuo_Wang8/publication/288699540/figure/fig9/AS:668373486686246@1536364065786/Illustration-of-AdaBoost-algorithm-for-creating-a-strong-classifier-based-on-multiple.png" width=500>

De forma resumida, as principais ideias por trás deste algoritmo são:

- O algoritmo cria e combina um conjunto de **modelos fracos** (em geral, stumps);
- Cada stump é criado **levando em consideração os erros do stump anterior**;
- Alguns dos stumps têm **maior peso de decisão** do que outros na predição final;

As classes no sklearn são:

- [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

- [AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor)

Note que não há muitos hiperparâmetros. O mais importante, que deve ser tunado com o grid/random search, é:

- `n_estimators` : o número de weak learners encadeados;

Além disso, pode também ser interessante tunar os hiperparâmetros dos weak learners. Isso é possível de ser feito, como veremos a seguir!


Como exemplo, vamos utilizar uma base de dados sobre [risco de crédito](https://www.kaggle.com/datasets/uciml/german-credit).

In [ ]:
import pandas as pd
df = pd.read_csv('german_credit_data.csv', index_col = 0)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
def pipe_pre_process():
    # Para facilitação de carregamento e transformação de dados
    
    # Carregamento de dados
    df = pd.read_csv('german_credit_data.csv')
    
    # Definição de features e target
    X = df.drop(columns='Risk')
    y = df["Risk"]
    
    # Particionamento dos conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
    
    #================================
    # Tratamento dos dados numéricos
    pipe_features_num = Pipeline([("input_num", SimpleImputer(strategy = "mean")), # substituindo valores nulos pela média
                                 ("scaler", StandardScaler())]) # padronização dos dados
    
    
    features_num = X_train.select_dtypes(include=np.number).columns.tolist() # selecionando apenas as colunas numéricas
    #==============================
    # Tratamento de features categóricas
    pipe_features_cat = Pipeline([("input_cat", SimpleImputer(strategy = "constant", fill_value = "unknown")), # dados categóricos faltantes
                                 ("onehot", OneHotEncoder())]) # transformação de dados categóricos
    
    features_cat = X_train.select_dtypes(exclude=np.number).columns.tolist() # selecionando apenas colunas categóricas
    #==============================
    # Aplicando as transformações nos nossos dados
    pre_processador = ColumnTransformer([("transf_num", pipe_features_num, features_num),
                                        ("transf_cat", pipe_features_cat, features_cat)])
    
    return X_train, X_test, y_train, y_test, pre_processador

In [ ]:
def metricas_classificacao(estimator):
    #=================
    print("\nMétricas da avaliação de treino:")
    
    y_pred_train = estimator.predict(X_train) # predição sobre os dados de treinamento
    
    print(confusion_matrix(y_train, y_pred_train)) # matriz de confusão
    
    ConfusionMatrixDisplay.from_predictions(y_train, y_pred_train)
    plt.show()
    
    print(classification_report(y_train, y_pred_train))
    
    #=================
    print("\nMétricas da avaliação de teste:")
    
    y_pred_test = estimator.predict(X_test) # predição sobre os dados de treinamento
    
    print(confusion_matrix(y_test, y_pred_test)) # matriz de confusão
    
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred_test)
    plt.show()
    
    print(classification_report(y_test, y_pred_test))

[AdaBoost classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X_train, X_test, y_train, y_test, pre_processador = pipe_pre_process()

In [ ]:
pre_processador

In [ ]:
# pipeline para o Adaboost
pipe_ab = Pipeline([("pre_processador", pre_processador),
                   ("ab", AdaBoostClassifier(random_state = 42))])

# por padrão, n_estimators = 50
pipe_ab.fit(X_train, y_train)

In [ ]:
metricas_classificacao(pipe_ab)

Usando $n_{estimators} = 150$ (apenas um "chute")

In [ ]:
# pipeline para o Adaboost
pipe_ab = Pipeline([("pre_processador", pre_processador),
                   ("ab", AdaBoostClassifier(random_state = 42, n_estimators = 150))])

# por padrão, n_estimators = 50
pipe_ab.fit(X_train, y_train)

In [ ]:
metricas_classificacao(pipe_ab)

# Exercício

Em grupos, trabalhem sobre o dataset 'german_credit_data' para tentar melhorar a performance do modelo com o adaboost. Que maneiras vocês enxergam de fazê-lo? Como implementá-las?